In [5]:
import numpy as np
import pandas as pd
import databricks.koalas as ks
from pyspark.sql import SparkSession

spark = None

In [17]:
if spark is not None:
    print('Stop previous session')
    spark.stop()
    
spark = SparkSession.builder.appName("tuto koalas") \
   .master("local[2]") \
   .config("spark.driver.extraJavaOptions", "-Xmx2g") \
   .getOrCreate()
spark

Stop previous session


In [10]:
listings_df = ks.read_csv("listings.csv") 
listings_df.head()

,id,station_id,commodity_id,supply,supply_bracket,buy_price,sell_price,demand,demand_bracket,collected_at
0,1,1,5,0,0,0,583,1810,2,1573354346
1,2,1,6,0,0,0,7195,163,2,1573354346
2,3,1,7,0,0,0,812,1087,2,1573354346
3,12,1,8,0,0,0,736,6159,3,1573354346
4,13,1,9,0,0,0,1260,1966,3,1573354346


In [11]:
commoditites_df = ks.read_json (r'commodities.json')
commoditites_df.head()

,average_price,buy_price_lower_average,category,category_id,ed_id,id,is_non_marketable,is_rare,max_buy_price,max_sell_price,min_buy_price,min_sell_price,name,sell_price_upper_average
0,419,178,"(1, Chemicals)",1,128049204,1,0,0,1114,2101,67,61,Explosives,1126
1,108,82,"(1, Chemicals)",1,128049202,2,0,0,137,198,28,27,Hydrogen Fuel,121
2,396,100,"(1, Chemicals)",1,128049203,3,0,0,600,1468,47,39,Mineral Oil,1208
3,438,123,"(1, Chemicals)",1,128049205,4,0,0,385,1626,66,58,Pesticides,822
4,459,198,"(2, Consumer Items)",2,128049241,5,0,0,526,963,95,78,Clothing,926


In [12]:
join_df = ks.merge(commoditites_df, listings_df, left_on='id',right_on='commodity_id',how='outer',suffixes=('_left','_right'))
join_df.head()

,average_price,buy_price_lower_average,category,category_id,ed_id,id_left,is_non_marketable,is_rare,max_buy_price,max_sell_price,min_buy_price,min_sell_price,name,sell_price_upper_average,id_right,station_id,commodity_id,supply,supply_bracket,buy_price,sell_price,demand,demand_bracket,collected_at
0,2100,1736,"(6, Machinery)",6,128049222,26,0,0,3078,3756,1587,1544,Crop Harvesters,2705,172,4,26,0,0,0,3121,364013,2,1574113085
1,2100,1736,"(6, Machinery)",6,128049222,26,0,0,3078,3756,1587,1544,Crop Harvesters,2705,205,5,26,0,0,0,2376,204410,1,1574125816
2,2100,1736,"(6, Machinery)",6,128049222,26,0,0,3078,3756,1587,1544,Crop Harvesters,2705,251,6,26,73805,2,2096,2022,0,0,1574116758
3,2100,1736,"(6, Machinery)",6,128049222,26,0,0,3078,3756,1587,1544,Crop Harvesters,2705,9443855,7,26,0,0,0,2635,0,1,1574113387
4,2100,1736,"(6, Machinery)",6,128049222,26,0,0,3078,3756,1587,1544,Crop Harvesters,2705,8762493,8,26,714888,2,2072,1999,0,0,1573753113


In [14]:
commoditites_df.explain()

== Physical Plan ==
*(3) Project [(_we0#297 - 1) AS __index_level_0__#295, average_price#266L, buy_price_lower_average#267L, category#268, category_id#269L, ed_id#270L, id#271L, is_non_marketable#272L, is_rare#273L, max_buy_price#274L, max_sell_price#275L, min_buy_price#276L, min_sell_price#277L, name#278, sell_price_upper_average#279L]
+- Window [row_number() windowspecdefinition(_w0#296L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS _we0#297], [_w0#296L ASC NULLS FIRST]
   +- *(2) Sort [_w0#296L ASC NULLS FIRST], false, 0
      +- Exchange SinglePartition
         +- *(1) Project [average_price#266L, buy_price_lower_average#267L, category#268, category_id#269L, ed_id#270L, id#271L, is_non_marketable#272L, is_rare#273L, max_buy_price#274L, max_sell_price#275L, min_buy_price#276L, min_sell_price#277L, name#278, sell_price_upper_average#279L, monotonically_increasing_id() AS _w0#296L]
            +- *(1) FileScan json [average_price#266L,buy_p

In [13]:
commoditites_df['category_name'] = commoditites_df.apply(lambda row: row.category['name'], axis = 1) 
commoditites_df.head()

PandasNotImplementedError: The method `pd.DataFrame.apply()` is not implemented yet.

In [ ]:
commoditites_df['test'] = commoditites_df.apply(lambda row: (row.category['name'], row['category_id']), axis = 1) 
commoditites_df.head()

In [ ]:
commoditites_df.test.unique()

In [ ]:
metals_only_df = join_df[join_df['category_id']==8]
metals_only_df.sort_values('sell_price', ascending=False)[['sell_price','name','station_id']].head()